In [10]:
from webScraper import sahibindenLib
from dataWorker import dataWorker

from bs4 import BeautifulSoup
from seleniumbase import Driver
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time, random, re
from sys import exit


In [ ]:
driver = Driver(uc=True)


In [ ]:
sahibindenLib.sahibindenInit(driver)



In [ ]:
sahibindenLib.sahibindenSearch(driver)

In [ ]:
current_url= driver.current_url
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

titles_raw = soup.find_all(class_="classifiedTitle")
titles = []
h_refs = []

for tt in titles_raw:
    print("----------------------------------------------")
    titles.append(tt.getText().strip())
    h_refs.append(current_url + tt.get("href"))

    print("current_url: " + tt.get("href"))

    print("Title: " + tt.getText().strip())
    print("Link: " + current_url + tt.get("href"))
    print("Link V2: " + "sahibinden.com" + tt.get("href"))
    print("----------------------------------------------")

In [128]:
import pandas as pd

def dataToPandas(title, link, square_meters, rooms, price, date, district):
        data = {
                "title": [title],
                "link": [link],
                "square_meters": [square_meters],
                "rooms": [rooms],
                "price": [price],
                "date": [date],
                "district": [district]
        }

        df = pd.DataFrame(data, index=[0])  # Pass index=[0] when creating the DataFrame
        df.to_csv("data.csv", mode="a", header=False)

def dataCSVControl(title, link, square_meters, rooms, price, date, district):
        df = pd.read_csv("data.csv")
        df = df.drop("Unnamed: 0", axis=1)
        df = df.drop_duplicates(subset=['link'], keep='first')
        df = df.reset_index(drop=True)
        df.to_csv("data.csv")

        if link not in df["link"].values:
                return True
        else:
                return False


def dataWorkerStart(title, link, square_meters, rooms, price, date, district):
       
        try:
                df = pd.read_csv("data.csv")
        except:
                df = pd.DataFrame(columns=["title", "link", "square_meters", "rooms", "price", "date", "district"])
                df.to_csv("data.csv")

        bool_CSV_Control = dataCSVControl(title, link, square_meters, rooms, price, date, district)
        if bool_CSV_Control:
                dataToPandas(title, link, square_meters, rooms, price, date, district)
                dataWorker.dataWorker(title, link, square_meters, rooms, price, date, district)


In [129]:
import pandas as pd
def sahibindenPostRecorder(driver):

    current_url= driver.current_url
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    titles_raw = soup.find_all(class_="classifiedTitle")
    specs_raw = soup.find_all(class_="searchResultsAttributeValue")
    price_raw = soup.find_all(class_="classified-price-container")
    dates_raw = soup.find_all(class_="searchResultsDateValue")
    districts_raw = soup.find_all(class_="searchResultsLocationValue")

    titles = []
    h_refs = []

    specs = [sp.getText().strip() for sp in specs_raw]
    rooms = specs[1::2]
    square_meters = specs[0::2]
    prices = [pr.getText().strip() for pr in price_raw]
    dates = [dt.getText().strip().replace("\n\n", " ") for dt in dates_raw]
    districts = [BeautifulSoup(str(ds), 'html.parser').get_text(separator='-').replace('/n', '-').replace(' / ', '-').strip() for ds in districts_raw]


    for tt in titles_raw:
        titles.append(tt.getText().strip())
        h_refs.append("https://www.sahibinden.com" + tt.get("href"))

    for i in range(len(titles)):
        inputs = driver.find_elements(By.CSS_SELECTOR, "input[type='text']")
        print("Title: " + titles[i])
        print("Link: " + h_refs[i])
        print("Square Meters: " + square_meters[i])
        print("Rooms: " + rooms[i])
        print("Price: " + prices[i])
        print("Date: " + dates[i])
        print("District: " + districts[i])
        print("--------------------------------------------------" + "\n")
    
        dataWorkerStart(titles[i], h_refs[i], square_meters[i], rooms[i], prices[i], dates[i], districts[i])
        
    
    return True

In [ ]:
sahibindenPostRecorder(driver)